# Demo
> **The purpose of this notebook is to demostrate**
> 1. How to conduct a feature space decomposition
> 2. How to visualize a decomposition
> 3. Qualitatively validate if the algorithm performs as expected

**This decomoposition only works on numerical values**

### Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification

from decomposition.var_clus import VarClus

### Demo 1: Instantiate the VarClus class

> **The parameters used in the constructor are**  

> **`n_split`**: Number of sub-clusters that every time a cluster is split into. Default 2  
**`max_eigenvalue`**: Eigenvalue threshold below which the decomposition will be stopped. Please note, the dataframe will be scaled during the process so each of the features will have variance == 1. Default 1   
**`max_tries`**: Number of max tries before the algorithm gives up. Default 3

> Besides the aformentioned properties, a key property is called Cluster. A Cluster object holds the information related to the below regards
1. What features are in the cluster
2. What are the parent clusters, if any
3. what are the child clusters, in any
4. Dataframe

In [2]:
demo1 = VarClus()

# Larger max_eigenvalue usually results in bigger and fewer child clusters
demo1 = VarClus(max_eigenvalue=5)

### Demo 2: Test on an arbitrary dataset

> Let's create some simple dataset to play with  
> We can leverage make_classification to make the dataset

In [3]:
n_features = 25
n_rows = 1e4

raw_df, _ = make_classification(n_samples=int(n_rows), 
                                n_features=n_features, 
                                n_informative=n_features,
                                n_redundant=0)

columns = ['feature_{}'.format(i) for i in range(n_features)]
demo2_df = pd.DataFrame(raw_df, columns=columns)

demo2 = VarClus(max_eigenvalue=1.35)
demo2.decompose(demo2_df)

decomposing cluster cluster-0
phase #1: NCS
phase #2: Search
assessing feature feature_1
current EV is 4.3428256693168885, new EV is 4.18940979140287
assessing feature feature_11
current EV is 4.3428256693168885, new EV is 4.338220555192775
assessing feature feature_12
current EV is 4.3428256693168885, new EV is 4.316820784176796
Number of max tries has been reached. Returning current result...
decomposing cluster cluster-0-0
phase #1: NCS
phase #2: Search
assessing feature feature_1
current EV is 3.900185244782042, new EV is 3.775658483399982
assessing feature feature_15
current EV is 3.900185244782042, new EV is 3.8335603311891058
assessing feature feature_16
current EV is 3.900185244782042, new EV is 3.7718957645795252
Number of max tries has been reached. Returning current result...
decomposing cluster cluster-0-1
phase #1: NCS
phase #2: Search
assessing feature feature_19
current EV is 3.12553763172441, new EV is 3.0336527231338604
assessing feature feature_24
current EV is 3.1255

> **The decomposition has a hierarchical structure, meaning the features of a child cluster are a subset of its parent cluster.
The whole algorithm can be described as below**

>> 1. Conducts PCA on current feature space. If the max eigenvalue is smaller than threshold,
    stop decomposition
2. Calculates the first N PCA components and assign features to these components based on
    absolute correlation from high to low. These components are the initial centroids of
    these child clusters.
3. After initial assignment, the algorithm conducts an iterative assignment called Nearest
    Component Sorting (NCS). Basically, the centroid vectors are re-computed as the first
    components of the child clusters and the algorithm will re-assign each of the feature
    based on the same correlation rule.
4. After NCS, the algorithm tries to increase the total variance explained by the first
    PCA component of each child cluster by re-assigning features across clusters

In [4]:
# Checkout the root_cluster
root_cluster = demo2.cluster

# Direct children of the root_cluster
child_clusters = root_cluster.children

# Direct parent of the root_cluster, if any
parent_clusters = root_cluster.parents

# root_cluster contains the original dataframe
root_cluster.dataframe.shape

(10000, 25)

In [5]:
# Print out the structure of the decomposition
demo2.print_cluster_structure()

cluster-0
|
|-----cluster-0-0
|     |
|     |-----cluster-0-0-0
|     |     |-----feature_1
|     |     |-----feature_15
|     |     |-----feature_16
|     |     |-----feature_17
|     |     |-----feature_18
|     |     |-----feature_2
|     |     |-----feature_20
|     |     |-----feature_21
|     |     |-----feature_22
|     |     |-----feature_23
|     |
|     |-----cluster-0-0-1
|           |-----feature_11
|           |-----feature_12
|           |-----feature_13
|           |-----feature_3
|           |-----feature_6
|
|-----cluster-0-1
      |
      |-----cluster-0-1-0
      |     |-----feature_19
      |     |-----feature_24
      |     |-----feature_4
      |     |-----feature_7
      |     |-----feature_8
      |
      |-----cluster-0-1-1
            |-----feature_0
            |-----feature_10
            |-----feature_14
            |-----feature_5
            |-----feature_9


### Demo 3: Test on a real dataset
> **UCI Machine Learning Repository - Wine Quality Data Set**  
https://archive.ics.uci.edu/ml/datasets/Wine+Quality


* **White wine first**

In [6]:
demo3_df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv', sep=';')

In [7]:
# Looks like quality is the dependent variable
demo3_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [8]:
demo3 = VarClus()
demo3.decompose(demo3_df.drop('quality', axis=1))

decomposing cluster cluster-0
phase #1: NCS
phase #2: Search
assessing feature alcohol
current EV is 4.723934521410767, new EV is 4.245067665175406
assessing feature chlorides
current EV is 4.723934521410767, new EV is 4.643244380951356
assessing feature density
current EV is 4.723934521410767, new EV is 4.079544622431165
Number of max tries has been reached. Returning current result...
decomposing cluster cluster-0-0
phase #1: NCS
phase #2: Search
assessing feature alcohol
current EV is 3.859524025576577, new EV is 3.691734857563688
assessing feature density
current EV is 3.859524025576577, new EV is 3.3128130916847427
assessing feature residual sugar
current EV is 3.859524025576577, new EV is 3.5363030396950004
Number of max tries has been reached. Returning current result...
decomposing cluster cluster-0-1
phase #1: NCS
phase #2: Search
assessing feature citric acid
current EV is 2.635513512191877, new EV is 2.6032183593434484
assessing feature fixed acidity
current EV is 2.63551351

In [9]:
demo3.print_cluster_structure()

cluster-0
|
|-----cluster-0-0
|     |
|     |-----cluster-0-0-0
|     |     |-----alcohol
|     |     |-----density
|     |     |-----residual sugar
|     |     |-----total sulfur dioxide
|     |
|     |-----cluster-0-0-1
|           |-----chlorides
|           |-----free sulfur dioxide
|
|-----cluster-0-1
      |
      |-----cluster-0-1-0
      |     |-----citric acid
      |     |-----fixed acidity
      |     |-----pH
      |
      |-----cluster-0-1-1
            |-----sulphates
            |-----volatile acidity


* **Red wine**

In [10]:
demo3_df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', sep=';')

In [11]:
# Looks like quality is the dependent variable
demo3_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [12]:
demo3 = VarClus()
demo3.decompose(demo3_df.drop('quality', axis=1))

decomposing cluster cluster-0
phase #1: NCS
phase #2: Search
assessing feature chlorides
current EV is 4.829692811123501, new EV is 4.754858095309795
assessing feature citric acid
current EV is 4.829692811123501, new EV is 4.217963436619195
assessing feature density
current EV is 4.829692811123501, new EV is 4.6323514547553835
Number of max tries has been reached. Returning current result...
decomposing cluster cluster-0-0
phase #1: NCS
phase #2: Search
assessing feature citric acid
current EV is 3.8229566024138113, new EV is 3.791360818746796
assessing feature density
current EV is 3.8229566024138113, new EV is 3.600357647184839
assessing feature fixed acidity
current EV is 3.8229566024138113, new EV is 3.2511781685143406
Number of max tries has been reached. Returning current result...
decomposing cluster cluster-0-1
phase #1: NCS
phase #2: Search
assessing feature free sulfur dioxide
Number of features is smaller than n_split, reducing n_split temporarily
current EV is 2.71143759600

In [13]:
# Similar structure
demo3.print_cluster_structure()

cluster-0
|
|-----cluster-0-0
|     |
|     |-----cluster-0-0-0
|     |     |-----citric acid
|     |     |-----density
|     |     |-----fixed acidity
|     |     |-----pH
|     |     |-----sulphates
|     |
|     |-----cluster-0-0-1
|           |-----chlorides
|           |-----volatile acidity
|
|-----cluster-0-1
      |
      |-----cluster-0-1-0
      |     |-----free sulfur dioxide
      |     |-----total sulfur dioxide
      |     |-----alcohol
      |
      |-----cluster-0-1-1
            |-----residual sugar
